<a href="https://colab.research.google.com/github/min02yam/project_2022_03/blob/main/H20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
install.packages("h2o")
library(h2o)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘RCurl’



----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit https://docs.h2o.ai

----------------------------------------------------------------------



Attaching package: ‘h2o’


The following objects are masked from ‘package:stats’:

    cor, sd, var


The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc




In [13]:
require(magrittr)
require(data.table)
require(dplyr)
require(readr)

Loading required package: magrittr



In [3]:
system("gdown --id 1PwFE91U4jhsO5gM4bWdnyvAClto3txOb")  # train
system("gdown --id 18elLaOJbV9rM-I0C-G1yswWKrc7ZV6yr")  # test

In [4]:
h2o.init(max_mem_size = "4g", nthreads = 5)


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpZuKmKl/file4755f5edf2/h2o_UnknownUser_started_from_r.out
    /tmp/RtmpZuKmKl/file47447dd0c/h2o_UnknownUser_started_from_r.err


Starting H2O JVM and connecting: .... Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         2 seconds 474 milliseconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.38.0.1 
    H2O cluster version age:    2 months and 17 days  
    H2O cluster name:           H2O_started_from_R_root_zqn464 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   4.00 GB 
    H2O cluster total cores:    2 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:    

In [5]:
DT= fread("train.csv")

In [6]:
h2o_dt = as.h2o(DT)

split_dt = h2o.splitFrame(h2o_dt, c(0.7), seed=49)
train =  h2o.assign(split_dt[[1]], "train")
test = h2o.assign(split_dt[[2]], "test")

  |======================================================================| 100%


In [7]:
y = "predicted_weight_g"
x = setdiff(names(train), y)

In [8]:
aml = h2o.automl(x = x, y = y,
                training_frame = train,
                max_models = 20,
                seed = 49)

  |======================================================================| 100%


In [9]:
lb = aml@leaderboard
print(lb, n = nrow(lb))

                                                  model_id      rmse       mse
1  StackedEnsemble_BestOfFamily_1_AutoML_1_20221207_130819  7.596724  57.71021
2     StackedEnsemble_AllModels_1_AutoML_1_20221207_130819  7.634013  58.27815
3                           GBM_4_AutoML_1_20221207_130819  7.876706  62.04250
4                           GBM_3_AutoML_1_20221207_130819  8.502536  72.29312
5                           GBM_5_AutoML_1_20221207_130819  8.795446  77.35987
6                           GBM_2_AutoML_1_20221207_130819  8.809905  77.61442
7          XGBoost_grid_1_AutoML_1_20221207_130819_model_2  8.914641  79.47083
8                       XGBoost_1_AutoML_1_20221207_130819  9.800329  96.04644
9                           XRT_1_AutoML_1_20221207_130819 10.181802 103.66909
10                      XGBoost_3_AutoML_1_20221207_130819 10.204073 104.12311
11         XGBoost_grid_1_AutoML_1_20221207_130819_model_1 10.497450 110.19646
12                      XGBoost_2_AutoML_1_20221207_

In [10]:
m = h2o.get_best_model(aml,criterion = "rmse")
h2o.performance(m, test) 

H2ORegressionMetrics: stackedensemble

MSE:  65.41603
RMSE:  8.088018
MAE:  5.15483
RMSLE:  NaN
Mean Residual Deviance :  65.41603


In [15]:
h2o.varimp(m)

Warning message:
“This model doesn't have variable importances”


In [11]:
test = fread("test.csv")
h2o_test = as.h2o(test)

  |======================================================================| 100%


In [14]:
fit = h2o.predict(object=m, newdata = h2o_test) %>% 
  as.data.table()

  |======================================================================| 100%


In [16]:
fit %>% head

predict
<dbl>
0.2634333
2.6737726
0.7841217
2.2267504
2.7831297
4.4431662


In [28]:
test$predicted_weight_g = fit$predict

In [18]:
test %>% head

case,DAT,내부온도관측치,내부습도관측치,co2관측치,ec관측치,시간당분무량,일간누적분무량,시간당백색광량,일간누적백색광량,시간당적색광량,일간누적적색광량,시간당청색광량,일간누적청색광량,시간당총광량,일간누적총광량,new_obs_time,predicted_weight_g
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,27.48973,61.37210,465.7229,1.037389,510.2554,6240.247,8728.198,102309.9,0,0,0,0,8728.198,102309.9,11.5,0.2634333
1,2,28.66880,63.14802,463.6613,1.034501,383.9642,5089.535,8729.874,102297.6,0,0,0,0,8729.874,102297.6,11.5,2.6737726
1,3,28.40740,61.04522,488.8831,1.033113,496.3237,6725.427,8734.386,102355.9,0,0,0,0,8734.386,102355.9,11.5,0.7841217
1,4,28.16375,64.35632,494.3944,1.033793,512.3987,6597.636,8732.839,102322.9,0,0,0,0,8732.839,102322.9,11.5,2.2267504
1,5,27.30222,66.82910,528.3493,1.038796,369.7646,4567.593,8734.515,102419.6,0,0,0,0,8734.515,102419.6,11.5,2.7831297
1,6,28.20056,64.31313,492.3361,1.027787,480.3571,5475.910,8734.515,102388.5,0,0,0,0,8734.515,102388.5,11.5,4.4431662


In [29]:
test[predicted_weight_g<0,predicted_weight_g := abs(predicted_weight_g) ]

In [30]:
#test %>% select(case, DAT, predicted_weight_g) -> tmp


for (i in 1:5){
  readr::write_csv(tmp %>% filter(case==i) %>% select(!case) , paste0('TEST_0',i,'.csv'))
}

#tmp %>% filter(case==1) %>% select(!case)
#tmp %>% filter(case==1) %>% select(!case)
#tmp %>% filter(case==1) %>% select(!case)
#tmp %>% filter(case==1) %>% select(!case)
#tmp %>% filter(case==1) %>% select(!case)




#readr::write_csv(tmp %>% filter(case==i) %>% select(!case) , path= paste0('TEST_0',i,'.csv'))